### **Setup** 

In [6]:
from dotenv import load_dotenv
import os
import requests

load_dotenv()

WEATHER_API_KEY = os.getenv("WEATHER_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
TUBE_API_KEY = os.getenv("TUBE_API_KEY")
SOURCE_CONTRY_CODE = "cl"

In [7]:
def load_prompt_from_file(file_path):
   with open(file_path, 'r') as file:
      return file.read()

### **Financial agent**


In [8]:
def get_indicador(indicator: str):
   """
   Get financial information.
   This function fetches data from the Mindicator API for a given indicator.
   It constructs the URL using the indicator name and makes a GET request to the API.
   The response is expected to be in JSON format, and the function returns the data if the request is successful.
   If the request fails, an exception is raised with an error message.
      
      Example:
         get_indicator("dolar")
      Example:
         get_indicator("uf")
      
      Params  
         indicator (str): The indicator to fetch.
      Returns:
         dict: The data for the indicator.
   """
   url = f"https://mindicador.cl/api/{indicator}"
   data = requests.get(url).json()
   if data.get("error") is None:
      return data
   else:
      raise Exception(f"Error fetching indicator {indicator}: {data['message']}")  

# print(get_indicator("dolar"))

In [9]:
from langchain_core.prompts import PromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain.tools import StructuredTool
from langchain.agents import (create_react_agent, AgentExecutor)

financial_llm = OllamaLLM(model="llama3", base_url="http://localhost:11434", temperature=0)

template = load_prompt_from_file("../prompts/financial.txt") 

prompt = PromptTemplate(
   template=template,
   input_variables=[
      "input", 
      "tools", 
      "tool_names",
      "agent_scratchpad", 
   ], 
)

tools_for_agent = [
   StructuredTool.from_function(
      name="get_indicador",
      func=get_indicador,
      description="""Get financial information.
   This function fetches data from the Mindicator API for a given indicator.
   It constructs the URL using the indicator name and makes a GET request to the API.
   The response is expected to be in JSON format, and the function returns the data if the request is successful.
   If the request fails, an exception is raised with an error message.
      
      Example:
         get_indicador(dolar)
      Example:
         get_indicador(uf)
      
      Params  
         indicator (str): The indicator to fetch.
      Returns:
         dict: The data for the indicator.
     """,
   )
]

financial_agent = create_react_agent(
   llm=financial_llm, 
   tools=tools_for_agent,
   prompt=prompt
)

financial_agent_executor = AgentExecutor(
   agent=financial_agent, 
   tools=tools_for_agent, 
   verbose=True,
)

### **Tavily search**

In [10]:
from langchain_community.tools.tavily_search import TavilySearchResults

# Web search
def get_location(location):
   """
   Get the location from the user.
   """
   search = TavilySearchResults(max_results=3)
   res = search.run(location)
   return res

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain.tools import StructuredTool
from langchain.agents import (create_react_agent, AgentExecutor)

template = load_prompt_from_file("prompts/weather.txt") 

llm = OllamaLLM(
   model="llama3", 
   base_url="http://localhost:11434", 
   temperature=0
)

prompt = PromptTemplate(
   template=template,
   input_variables=["input", "tools", "tool_names", "agent_scratchpad"], 
)

tools_for_agent = [
   StructuredTool.from_function(
      name="get_location",
      func=get_location,
      description="Get the location for a city.",
   )
]

agent = create_react_agent(
   llm=llm, 
   tools=tools_for_agent,
   prompt=prompt
)

agent_executor = AgentExecutor(
   agent=agent, 
   tools=tools_for_agent, 
   verbose=True,
)

# Primer agente - obtiene los datos de la localidad
weather_result = agent_executor.invoke(
   input={"input": "Quiero saber donde esta Santiago de Chile"}
)

print(weather_result["output"])

### **Notice agent** 

In [11]:
def get_notices():
   """
   get news from the api.
   the response is expected to be in json format, and the function returns the data if the request is successful.
   if the request fails, an exception is raised with an error message.
      
      example:
         get_notices()
      
      returns:
         dict: the data for the notice. 
   """
   url = f"https://api.apitube.io/v1/news/everything?api_key={TUBE_API_KEY}&source.country.code={SOURCE_CONTRY_CODE}"
   data = requests.get(url).json()
   if data.get("error") is None:
      return data
   else:
      raise Exception("Error fetching")  

In [12]:
from langchain_core.prompts import PromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain.tools import StructuredTool
from langchain.agents import (create_react_agent, AgentExecutor)

notice_llm = OllamaLLM(model="llama3", base_url="http://localhost:11434", temperature=0)

template = load_prompt_from_file("../prompts/notice.txt") 

prompt = PromptTemplate(
   template=template,
   input_variables=[
      "input", 
      "tools", 
      "tool_names",
      "agent_scratchpad", 
   ], 
)

tools_for_agent = [
   StructuredTool.from_function(
      name="get_notices",
      func=get_notices,
      description="""
      get news from the api.
      the response is expected to be in json format, and the function returns the data if the request is successful.
      if the request fails, an exception is raised with an error message.
      
      example:
         get_notices()
      
      returns:
         dict: the data for the notice.
      """,
   )
]

notice_agent = create_react_agent(
   llm=notice_llm, 
   tools=tools_for_agent,
   prompt=prompt
)

notice_agent_executor = AgentExecutor(
   agent=notice_agent, 
   tools=tools_for_agent, 
   verbose=True,
   handle_parsing_errors=True,
)

### **Weather Agent**

In [ ]:
def get_weather_test():
   """
   Get the weather from the api.
   the response is expected to be in json format, and the function returns the data if the request is successful.
   if the request fails, an exception is raised with an error message.
      
      example:
         get_weather_test()
      
      returns:
         dict: the data for the weather. 
   """
   url = "https://api.openweathermap.org/data/2.5/weather"
   
   params = {
      "q": "Santiago",
      "appid": WEATHER_API_KEY,
      "lang": "es",
      "lat": 7.3891,
      "lon": 5.9845,
   }
   data = requests.get(url, params=params).json()
   print(data)
   if data.get("error") is None:
      return data
   else:
      raise Exception("Error fetching")
   
get_weather_test()


{'coord': {'lon': -5.9761, 'lat': 37.3824}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'cielo claro', 'icon': '01n'}], 'base': 'stations', 'main': {'temp': 289.21, 'feels_like': 288.73, 'temp_min': 287.63, 'temp_max': 289.82, 'pressure': 1018, 'humidity': 71, 'sea_level': 1018, 'grnd_level': 1015}, 'visibility': 10000, 'wind': {'speed': 2.24, 'deg': 356, 'gust': 3.13}, 'clouds': {'all': 0}, 'dt': 1745463687, 'sys': {'type': 2, 'id': 2011488, 'country': 'ES', 'sunrise': 1745473025, 'sunset': 1745521597}, 'timezone': 7200, 'id': 2510911, 'name': 'Seville', 'cod': 200}


{'coord': {'lon': -5.9761, 'lat': 37.3824},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'cielo claro',
   'icon': '01n'}],
 'base': 'stations',
 'main': {'temp': 289.21,
  'feels_like': 288.73,
  'temp_min': 287.63,
  'temp_max': 289.82,
  'pressure': 1018,
  'humidity': 71,
  'sea_level': 1018,
  'grnd_level': 1015},
 'visibility': 10000,
 'wind': {'speed': 2.24, 'deg': 356, 'gust': 3.13},
 'clouds': {'all': 0},
 'dt': 1745463687,
 'sys': {'type': 2,
  'id': 2011488,
  'country': 'ES',
  'sunrise': 1745473025,
  'sunset': 1745521597},
 'timezone': 7200,
 'id': 2510911,
 'name': 'Seville',
 'cod': 200}

In [32]:
# Función para consultar el clima

# Weather TOOL
def get_weather(input_str: str) -> dict:
   """
      Search for the current weather in a given city.
      
      Params:
         input_str (str): A string containing city and optional coordinates either as JSON or in format "city='Madrid', lat=40.4168, lon=-3.7038, part=''"
         
      Returns:
         dict: The weather data for the given city.
   """
   try:
      import json
      print("Weather function called")
      print(f"Input string: {input_str}")
      # Parse the input to extract parameters
      params = {}
      
      # Check if input is already a dict
      if isinstance(input_str, dict):
         params = input_str
      # Check if input is a JSON string
      elif isinstance(input_str, str):
         input_str = input_str.strip()
         if input_str.startswith('{') and input_str.strip().endswith('}'):
            try:
               params = json.loads(input_str)
            except json.JSONDecodeError as e:
               print(f"Error parsing JSON input: {e}")
               return {"error": f"Invalid JSON format: {str(e)}"}
         else:
            # Parse key-value pairs from string format
            parts = input_str.split(',')
            for part in parts:
               part = part.strip()
               if '=' in part:
                  key_value = part.split('=', 1)  # Split on first = only
                  if len(key_value) == 2:
                     key = key_value[0].strip()
                     value = key_value[1].strip().strip("'\"")
                     # Convert numeric values to proper types
                     try:
                        if '.' in value and any(c.isdigit() for c in value):
                           params[key] = float(value)
                        elif value.isdigit():
                           params[key] = int(value)
                        else:
                           params[key] = value
                     except:
                        params[key] = value
      
      # Extract parameters with default values
      city = params.get('city', '')
      lat = params.get('lat', None)
      lon = params.get('lon', None)
      part = params.get('part', '')
      units = params.get('units', 'metric')  
      lang = params.get('lang', 'es')
      
      # Validate required parameters
      if not city and (lat is None or lon is None):
         return {"error": "Missing required parameters: either city or both lat and lon must be provided"}
      
      # Construct API request
      url = "https://api.openweathermap.org/data/2.5/weather"
      api_params = {
         "q": city,
         "lat": lat,
         "lon": lon,
         "part": part,
         "appid": WEATHER_API_KEY,
         "units": units,
         "lang": lang,
      }
      
      # Set either city or coordinates
      if lat is not None and lon is not None:
         try:
            api_params["lat"] = float(lat)
            api_params["lon"] = float(lon)
            print(f"Using coordinates: Latitude: {lat}, Longitude: {lon}")
         except (ValueError, TypeError):
            return {"error": "Invalid coordinates: lat and lon must be numeric values"}
      else:
         api_params["q"] = city
         print(f"Using city name: {city}")
      
      # Make the request
      print(f"Making API request to {url} with params: {api_params}")
      request = requests.get(url, params=api_params)
      request.raise_for_status()
      
      # Return the JSON response
      return request.json()
      
   except requests.exceptions.RequestException as e:
      print(f"Error fetching weather data: {e}")
      return {"error": f"API request failed: {str(e)}"}
   except Exception as e:
      print(f"Error processing input: {e}")
      return {"error": f"Could not process input: {str(e)}"}

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_ollama.llms import OllamaLLM
from langchain.tools import StructuredTool
from langchain.agents import (create_react_agent, AgentExecutor)

weather_llm = OllamaLLM(model="llama3", base_url="http://localhost:11434", temperature=0)

template = load_prompt_from_file("../prompts/weather.txt") 

prompt = PromptTemplate(
   template=template,
   input_variables=["question", "tools", "tool_names", "agent_scratchpad"], 
)

tools_for_agent = [
   StructuredTool.from_function(
      name="get_weather",
      func=get_weather,
      description="Get the weather data for a city. Required parameters: city (name of the city), optional: lat (latitude), lon (longitude), part (parts to exclude). If you don't know the city, don't use this tool.",
   )
]

weather_agent = create_react_agent(
   llm=weather_llm, 
   tools=tools_for_agent,
   prompt=prompt
)

weather_agent_executor = AgentExecutor(
   agent=weather_agent, 
   tools=tools_for_agent, 
   verbose=True,
)

   


### **INTERPRETER AGENT**

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = load_prompt_from_file("../prompts/interpreter.txt")
 
interpreter_prompt = PromptTemplate(
   template=template,
   input_variables=["raw_response"],
)

llm_interpreter = OllamaLLM(
   model="llama3", 
   base_url="http://localhost:11434",
   temperature=0,
)
parser = StrOutputParser()
interpreter_chain = interpreter_prompt | llm_interpreter



### **Playground** 

In [34]:

def query_weather(user_question):
   # Primer agente - obtiene los datos del clima
   weather_result = weather_agent_executor.invoke(
      input={"input": user_question}
   )
   
   # Segundo agente - interpreta y presenta los datos
   friendly_response = interpreter_chain.invoke(
      {"raw_response": weather_result["output"]}
   )
   return friendly_response
   

# Ejemplo de uso
user_query = "¿Qué tiempo hace hoy en Santiago?"
friendly_weather_response = query_weather(user_query)
print(f"Consulta: {user_query}\n\nRespuesta: {friendly_weather_response}")




> Entering new AgentExecutor chain...
A Spanish question! Let's see...

Thought: The user is asking about the weather in Santiago today.

Action: get_weather

Action Input:
{
    "city": "Santiago",
    "lat": -33.4489,  # Optional
    "lon": -70.6693,  # Optional
    "part": ""       # Optional
}
Weather function called
Input string: {
    "city": "Santiago",
    "lat": -33.4489,  # Optional
    "lon": -70.6693,  # Optional
    "part": ""       # Optional
}

Error parsing JSON input: Expecting property name enclosed in double quotes: line 3 column 23 (char 48)
{'error': 'Invalid JSON format: Expecting property name enclosed in double quotes: line 3 column 23 (char 48)'}It looks like there's an issue with the `get_weather` tool. The error message suggests that the input JSON is not properly formatted.

Let me try again, this time making sure to format the Action Input correctly:

Action: get_weather

Action Input:
{
    "city": "Santiago",
    "lat": -33.4489,
    "lon": -70.6693,
  

In [ ]:
def query_indicator(user_question):
   # Primer agente - obtiene los datos del indicador
   indicator_result = financial_agent_executor.invoke(
      input={"input": user_question}
   )
   
   # Segundo agente - interpreta y presenta los datos
   friendly_response = interpreter_chain.invoke(
      {"raw_response": indicator_result["output"]}
   )
   
   return friendly_response

# Ejemplo de uso
user_query = "¿Cuál es el valor del uf hoy?"
friendly_indicator_response = query_indicator(user_query)
print(f"Consulta: {user_query}\n\nRespuesta: {friendly_indicator_response}")


In [12]:
def query_notice(user_question):
   # Primer agente - obtiene los datos del indicador
   notice_result = notice_agent_executor.invoke(
      input={"input": user_question}
   )
   
   # Segundo agente - interpreta y presenta los datos
   friendly_response = interpreter_chain.invoke(
      {"raw_response": notice_result["output"]}
   )
   
   return friendly_response

# Ejemplo de uso
user_query = "¿Cuáles son las noticias de hoy?"
friendly_notice_response = query_notice(user_query)
print(f"Consulta: {user_query}\n\nRespuesta: {friendly_notice_response}")



> Entering new AgentExecutor chain...
Thought: I should get the news for today.

Action: get_notices
Action Input: None{'status': 'ok', 'limit': 10, 'path': 'https://api.apitube.io/v1/news/everything?api_key=api_live_D5HTMscr9FsXCVfvRISMoGMd10MEJBoHw9EPL3nZJ1AjvT&source.country.code=cl', 'page': 1, 'has_next_pages': True, 'next_page': 'https://api.apitube.io/v1/news/everything?api_key=api_live_D5HTMscr9FsXCVfvRISMoGMd10MEJBoHw9EPL3nZJ1AjvT&page=2&source.country.code=cl', 'next_page_cursor': 'AoIqMzAyNjYyMzkyOXCz4KPslQM=', 'has_previous_page': False, 'previous_page': '', 'export': {'json': 'https://api.apitube.io/v1/news/everything?api_key=api_live_D5HTMscr9FsXCVfvRISMoGMd10MEJBoHw9EPL3nZJ1AjvT&export=json&source.country.code=cl', 'xlsx': 'https://api.apitube.io/v1/news/everything?api_key=api_live_D5HTMscr9FsXCVfvRISMoGMd10MEJBoHw9EPL3nZJ1AjvT&export=xlsx&source.country.code=cl', 'csv': 'https://api.apitube.io/v1/news/everything?api_key=api_live_D5HTMscr9FsXCVfvRISMoGMd10MEJBoHw9EPL3n

KeyboardInterrupt: 